In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random

Graph $(V, E)$ with nodes $v\subset\mathbb{N}$ as subsets of integers
Strategy for merging:
1. Draw a set of edges $F\subseteq E$ somehow
2. Identify connected components $\mathcal{C}=\{C_1,\dotsc, C_k\}$ of $(V, F)$. They form the new set of nodes
3. Compute new adjacency matrix $A^\prime$ by summing entries of $A$ blockwise 

#### Rule of thumb for choosing temperature $\tau$:
If there are $m$ edges in an unweighted graph, the amount of chosen edges is distrubted as Bin($e$, $n^{-\tau}$). Therefore, the expected number of chosen edges is $n^{1-\tau}$, and a choice of 
$$
\tau=\frac{\log(2)}{\log(m)}
$$
leads to $m/2$ in expectation.

In [23]:
class FindCuts(object):
    def __init__(self, A, sample_fn, partition_fn):
        self.A = A
        self.V = [{i} for i in range(nx.number_of_nodes(G))]
        self.sample_fn = sample_fn
        self.partition_fn = partition_fn
    
    def _merge_nodes(self, V, component):
        """ Merge the sets of V that are indexed in component."""
        merged_node = set({})
        for idx in component:
            merged_node = merged_node.union(V[idx])
        return merged_node
    
    def _merge_edges(self, V, A, A_sampled):
        """ Merge edges in A_sampled to obtain a new graph."""
        G_sampled = nx.from_numpy_array(A_sampled)
        connected_components = list(nx.connected_components(G_sampled))
        V_coarse = [self._merge_nodes(V, component) for component in connected_components]
        n_coarse = len(V_coarse)
        A_coarse = np.zeros((n_coarse, n_coarse))
        for i in range(n_coarse):
            for j in range(i+1, n_coarse):
                for v in connected_components[i]:
                    for w in connected_components[j]:
                        A_coarse[i, j] += A[v, w] 
        A_coarse += A_coarse.T
        return V_coarse, A_coarse
    
    def get_cuts(self, T):
        """ T: number of partitions"""
        V = self.V
        A = self.A
        for t in range(T):
            A_sampled = self.sample_fn(A)
            V, A = self._merge_edges(V, A, A_sampled)
            P = self.partition_fn(V, A)
            print(P)
        return

In [15]:
def sample_edges(A, temperature=.7):
    """ Sample each edge with a probability, scaled by the temperature.
    Returns new edges in form of adjacency matrix"""
    edges_idx = np.triu_indices_from(A, k=1)
    edge_probs = (A[edges_idx] / A[edges_idx].sum())**temperature
    sampled_edges = np.random.binomial(n=1, p=edge_probs)
    is_sampled = (sampled_edges == 1)
    sampled_edges_idx = (edges_idx[0][is_sampled], edges_idx[1][is_sampled])
    A_sampled = np.zeros_like(A)
    A_sampled[sampled_edges_idx] = 1
    A_sampled += A_sampled.T
    return A_sampled

def partition(V, A):
    return np.sort([len(v) for v in V])[::-1]

### Example: Stochastical Block model

In [17]:
G = nx.stochastic_block_model(sizes=[50, 50], p=[[.8, .1],
                                                 [.1, .8]])
A = nx.to_numpy_array(G)
V = [{i} for i in range(A.shape[0])]

print(f'Original graph: {len(V)} nodes')
print(V)
print(A)

Original graph: 100 nodes
[{0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}, {10}, {11}, {12}, {13}, {14}, {15}, {16}, {17}, {18}, {19}, {20}, {21}, {22}, {23}, {24}, {25}, {26}, {27}, {28}, {29}, {30}, {31}, {32}, {33}, {34}, {35}, {36}, {37}, {38}, {39}, {40}, {41}, {42}, {43}, {44}, {45}, {46}, {47}, {48}, {49}, {50}, {51}, {52}, {53}, {54}, {55}, {56}, {57}, {58}, {59}, {60}, {61}, {62}, {63}, {64}, {65}, {66}, {67}, {68}, {69}, {70}, {71}, {72}, {73}, {74}, {75}, {76}, {77}, {78}, {79}, {80}, {81}, {82}, {83}, {84}, {85}, {86}, {87}, {88}, {89}, {90}, {91}, {92}, {93}, {94}, {95}, {96}, {97}, {98}, {99}]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]


In [28]:
findCuts = FindCuts(A=A, sample_fn=sample_edges, partition_fn=partition)
findCuts.get_cuts(T=30)

[4 4 3 3 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1]
[5 5 4 4 3 3 3 3 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[7 6 5 4 3 3 3 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[9 8 5 4 3 3 3 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[9 8 5 5 4 3 3 3 3 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[9 9 9 5 4 3 3 3 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[24  9  7  4  4  4  3  2  2  2  2  2  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1]
[44  7  7  4  2  2  2  2  2